In [ ]:
import pandas as pd
import tqdm.auto as tqdm

complete_to_two_digits = lambda x: str(x).zfill(2) if x < 10 else str(x)

def read_csv_file(user_id, video_id):
    filepath = f'../sensory/raw/{video_id}_user{user_id}_raw.csv'
    #print(filepath)
    df = pd.read_csv(filepath)
    
    return df

csv_data = []
video_ids = ['coaster', 'coaster2', 'diving', 'drive', 'game', 'landscape', 'pacman', 'panel', 'ride', 'sport']
for user_id in range(1, 51):
    user_data = []
    for video_id in video_ids:

        df = read_csv_file(complete_to_two_digits(user_id), video_id)
 
        df['user_id'] = user_id # 添加标签（用户ID）
        df['video_id'] = video_id # 添加video_id字段
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        df['timestamp_floor'] = pd.to_datetime(df['timestamp'], unit='s').dt.floor('100ms')
        user_data.append(df)

    user_data_combined = pd.concat(user_data, ignore_index=True)
    csv_data.append(user_data_combined)

csv_data = pd.concat(csv_data, ignore_index=True)


In [ ]:
csv_data

In [ ]:
def append_to_file(input_file, output_file, additional_content):
    with open(input_file, 'r') as f:
        original_content = f.read()

    new_content = original_content + '\n' + additional_content

    with open(output_file, 'w') as f:
        f.write(new_content)


In [ ]:
def initTimeNetData(data, label, step):
    append_string = ""
    for i in range(0, len(label), step):
        group = label[i:i+step]
        if len(set(group)) == 1:
            temp_transposed_matrix = np.transpose(data[i:i+step])
            append_string += ":".join([",".join(map(str, row)) for row in temp_transposed_matrix])
            append_string += ":" + str(group[0]) + "\n"

    return append_string

In [ ]:
from datetime import timedelta
import numpy as np
import random
from tqdm.auto import tqdm

def process_data(csv_data, test_video_id):
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    for user_id, user_data in tqdm(csv_data.groupby('user_id')):
        for video_id, video_data in user_data.groupby('video_id'):
            for _, floor_data in video_data.groupby(video_data['timestamp_floor']):
                floor_data = floor_data[['rawTX', 'rawTY', 'rawTZ', 'rawYaw', 'rawPitch', 'rawRoll']]
                if video_id == test_video_id:
                    test_data.append(round(floor_data.mean(), 5).values)
                    test_labels.append(user_id)
                else:
                    train_data.append(round(floor_data.mean(), 5).values)
                    train_labels.append(user_id)

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [ ]:
i = np.Inf
for video_id in video_ids:
    if video_id == 'coaster':
        i = 0
    elif video_id == 'coaster2':
        i = 1
    elif video_id == 'diving':
        i = 2
    elif video_id == 'drive':
        i = 3
    elif video_id == 'game':
        i = 4
    elif video_id == 'landscape':
        i = 5
    elif video_id == 'pacman':
        i = 6
    elif video_id == 'panel':
        i = 7
    elif video_id == 'ride':
        i = 8
    elif video_id == 'sport':
        i = 9
    print(f'i: {i}, video_id: {video_id} begin.')
    train_data, train_labels, test_data, test_labels = process_data(csv_data, video_id)
    print('train_data.shape: {}'.format(train_data.shape))
    print('train_labels.shape: {}'.format(train_labels.shape))
    print('test_data.shape: {}'.format(test_data.shape))
    print('test_labels.shape: {}'.format(test_labels.shape))
    append_train_string = initTimeNetData(train_data, train_labels, 10)
    append_to_file('template.ts', f'UserIdentify{i}_TRAIN.ts', append_train_string)
    append_test_string = initTimeNetData(test_data, test_labels, 10)
    append_to_file('template.ts', f'UserIdentify{i}_TEST.ts', append_test_string)
    print(f'i: {i}, video_id: {video_id} finished.')